In [27]:
# Commented out IPython magic to ensure Python compatibility.
# Colab and system related
import os
import time
from google.colab import drive
import sys
drive.mount('/content/drive')

#Nifty
if os.path.isdir('nifty') == False:
    !git clone https://gitlab.mpcdf.mpg.de/ift/nifty.git
sys.path.append('/content/nifty')
import nifty6 as ift
sys.path.append('/content/drive/My Drive/masterthesis/')
from operators.tensorflow_operator import TensorFlowOperator
sys.path.append('/content/drive/My Drive/masterthesis/helper_functions/')
from helper_functions import clear_axis, corner_mask, half_mask, checkerboard_mask, no_mask, gaussian, get_cmap, info_text
def get_noise(noise_level):
    N_ift = ift.ScalingOperator(position_space, noise_level)
    n = N_ift.draw_sample_with_dtype(dtype=np.float64)
    return N_ift, n  # N respresents the noise operator (diagnonal covariance), n represents acutal sampled noise values
# Tensorflow
import tensorflow as tf

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm 

# %matplotlib inline
mpl.rcParams['figure.dpi']= 150
mpl.rcParams['font.size'] = 9.0

# Numerics
import random
import numpy as np
from sklearn.neighbors import KernelDensity
from scipy.stats import multivariate_normal

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# Load MNIST Dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [29]:
x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))
input_shape = (784,)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255


In [30]:
Input = tf.keras.layers.Input(shape=784)
h1 = tf.keras.layers.Dense(512, activation='elu')(Input)
h2 = tf.keras.layers.Dense(256, activation='elu')(h1)
h3 = tf.keras.layers.Dense(128, activation='elu')(h2)
h4 = tf.keras.layers.Dense(64, activation='elu')(h3)
h5 = tf.keras.layers.Dense(10, activation='linear')(h4)
encoded = tf.keras.activations.softmax(h5)
model = tf.keras.Model(Input, encoded)

In [31]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x_train,y_train, epochs=20)

Train on 60000 samples
Epoch 1/20
60000/60000 [==============================] - 12s 200us/sample - loss: 0.4876 - acc: 0.8204
Epoch 2/20
60000/60000 [==============================] - 12s 194us/sample - loss: 0.3833 - acc: 0.8580
Epoch 3/20
60000/60000 [==============================] - 12s 197us/sample - loss: 0.3478 - acc: 0.8708
Epoch 4/20
60000/60000 [==============================] - 12s 198us/sample - loss: 0.3227 - acc: 0.8805
Epoch 5/20
60000/60000 [==============================] - 12s 195us/sample - loss: 0.3053 - acc: 0.8864
Epoch 6/20
60000/60000 [==============================] - 12s 201us/sample - loss: 0.2881 - acc: 0.8919
Epoch 7/20
60000/60000 [==============================] - 12s 200us/sample - loss: 0.2783 - acc: 0.8972
Epoch 8/20
60000/60000 [==============================] - 12s 204us/sample - loss: 0.2634 - acc: 0.9019
Epoch 9/20
60000/60000 [==============================] - 12s 199us/sample - loss: 0.2535 - acc: 0.9035
Epoch 10/20
60000/60000 [================

In [32]:
model.evaluate(x_test, y_test)

[0.3624039155483246, 0.8889]

In [33]:
model.save('drive/My Drive/masterthesis/modeling/NNs/Fashion-MNIST/pretrained_classifier/Classifier', save_format='tf')

INFO:tensorflow:Assets written to: drive/My Drive/masterthesis/modeling/NNs/Fashion-MNIST/pretrained_classifier/Classifier/assets


In [34]:
print(x_test[10, :].shape)
print(x_test.shape)
#a = (loss_fn(y_test[10], model.predict(np.array([x_test[10, :]]))))
print(y_test[20])
print(tf.keras.backend.get_value(model.predict(np.array([x_test[20, :]]))))
print(model.layers[-1].output)

(784,)
(10000, 784)
2
[[2.23576339e-04 6.33494505e-13 9.99600708e-01 1.79803639e-09
  7.49452420e-06 4.69729899e-21 1.68233819e-04 6.38375645e-16
  1.07188806e-10 6.74173458e-18]]
Tensor("Softmax_3:0", shape=(?, 10), dtype=float32)
